### Analysing the communication between Enron employees and Arthur Andersen LLP, the accounting firm associated with Enron Corporation.<br>
U.S. Justice Department indicted Arthur Anderson in March 14, 2002 for obstruction of justice. Later in 2002, the firm voluntarily surrendered its licenses to practice as Certified Public Accountants in the United States after being found guilty of criminal charges relating to the firm's handling of the auditing of Enron.

Key analysis:

- Our key analysis will be between January 2000 few months before Enron stocks sky rocketed and January 2002 when Enron ended partnership with the accounting firm, days after SEC widens its investigation to Arthur Anderson.
- Map enron employees emails with reference Arthur Anderson.<br>
- Identify months on which Arthur Anderson was referred on emails suspiciously.

In [1]:
import nltk, os, glob, email, datetime, re
import email.utils
import time
import datetime

directory = 'data/enron/enron_mail_20150507/maildir/'

In [12]:
user_list = []

employee_name_map = dict()

frequent_senders = dict()
monthly_mails = dict()
anderson_employees_contacted = []

#identifying list of users
for users in os.walk(directory):
    user_list = users[1]
    break
#print(user_list)  

print("Traversing through each user folders...")
for user in user_list:
    print(user)
    user_folder = directory + user
    
    #identifying sub directories under user folder
    for d in os.walk(user_folder):
        sub_dir = d[1]
        break

    # identifying folders with word sent in it
    sent_folders = []
    if sub_dir:
        for x in sub_dir:
            if x.startswith('sent'):
                sent_folders.append(x)
   
    #iterating through sent folders
    #print('Iterating through sent folders')
    for sent_folder in sent_folders:
        sent_folder_files = user_folder +'/'+sent_folder+'/*'
        root, dirs, sent_folder_filenames = next(os.walk(user_folder +'/'+sent_folder))

        for filename in sent_folder_filenames:
            fname = user_folder +'/'+sent_folder+'/'+filename
            with open(fname,'r',encoding='utf-8',errors='ignore') as file:
                #whole email
                message = email.message_from_string(file.read())

                #keys
                #print(message.keys())
                
                 #from
                if 'From' in message.keys():
                    from_email = message.get('From')
                if 'X-From' in message.keys():
                    from_name = message.get('X-From')
                    
                #date
                #initial analysis shows date in PDT/PST
                sdate = message.get('Date')
                # Fri, 16 Feb 2001 09:26:00 -0800 (PST)
                #datetime_object = datetime.datetime.strptime(sdate, '%b %d %Y %I:%M%p')
                #print(sdate.datetime())
                pdate = email.utils.parsedate(sdate)
                make_time = time.mktime(pdate)
                parse_date = datetime.datetime.fromtimestamp(make_time)
                #print(parse_date) 1999-02-16 06:39:00
                #Checking for month entry in dictionary
                date_entry = parse_date.strftime("%B")+'-'+parse_date.strftime("%Y")
                    
                #employee name map    
                if from_email not in employee_name_map.keys():
                    employee_name_map[from_email] = from_name    
                    
                #to
                if 'To' in message.keys():
                    to_emails = message.get('To')
                    to_email_list = to_emails.split(',')

                    flag = 0;
                    for to_email in to_email_list:
                        #check if email is directed to anderson emaployees or not 
                        if 'arthuranderson' in to_email:
                            to_email_stripped = to_email.replace('\n','').replace('\t','').strip()
                            if to_email_stripped not in anderson_employees_contacted:
                                anderson_employees_contacted.append(to_email_stripped)
                            flag = 1
                        
                        #if not in to_email, check if Arthur Anderson is mentioned in email or not
                        if flag == 0:
                            #body
                            if not message.is_multipart():
                                body = message.get_payload()
    
                            if 'arthur anderson' in body.lower():
                                flag = 1;
                
                        if flag == 1:
                            #adding email count to frequent_senders dictionary
                            if from_email not in frequent_senders:
                                frequent_senders[from_email] = 1
                            else:
                                frequent_senders[from_email] += 1
                            
                            #adding count to corresponding month
                            if date_entry in monthly_mails.keys():
                                monthly_mails[date_entry] = monthly_mails[date_entry] + 1
                            else:
                                monthly_mails[date_entry] = 1

                                
print("\nArthur Anderson firm employees contacted:")                     
for b in anderson_employees_contacted:
    print(b)
                    
print("\nTop 5 Employees who contacted Arthur Anderson firm:")    
sorted_frequent_senders = [(k, frequent_senders[k]) for k in sorted(frequent_senders, key=frequent_senders.get, reverse=True)]
for b in sorted_frequent_senders[:5]:
    print(str(b[1])+' - '+b[0])
    
print("\nTop 5 Months Arthur Anderson firm was contacted:")
sorted_monthly_mails = [(k, monthly_mails[k]) for k in sorted(monthly_mails, key=monthly_mails.get, reverse=True)]
for b in sorted_monthly_mails[:5]:
    print(str(b[1])+' - '+b[0])
print("\nApril 2000 - 3 months before Jeffrey Skilling resigns as CEO, and Kenneth Lay becomes CEO again.")

Traversing through each user folders...
allen-p
arnold-j
arora-h
badeer-r
bailey-s
bass-e
baughman-d
beck-s
benson-r
blair-l
brawner-s
buy-r
campbell-l
carson-m
cash-m
causholli-m
corman-s
crandell-s
cuilla-m
dasovich-j
davis-d
dean-c
delainey-d
derrick-j
dickson-s
donoho-l
donohoe-t
dorland-c
ermis-f
farmer-d
fischer-m
forney-j
fossum-d
gang-l
gay-r
geaccone-t
germany-c
gilbertsmith-d
giron-d
griffith-j
grigsby-m
guzman-m
haedicke-m
hain-m
harris-s
hayslett-r
heard-m
hendrickson-s
hernandez-j
hodge-j
holst-k
horton-s
hyatt-k
hyvl-d
jones-t
kaminski-v
kean-s
keavey-p
keiser-k
king-j
kitchen-l
kuykendall-t
lavorato-j
lay-k
lenhart-m
lewis-a
linder-e
lokay-m
lokey-t
love-p
lucci-p
maggi-m
mann-k
martin-t
may-l
mccarty-d
mcconnell-m
mckay-b
mckay-j
mclaughlin-e
merriss-s
meyers-a
mims-thurston-p
motley-m
neal-s
nemec-g
panus-s
parks-j
pereira-s
perlingiere-d
phanis-s
pimenov-v
platter-p
presto-k
quenet-j
quigley-d
rapp-b
reitmeyer-j
richey-c
ring-a
ring-r
rodrique-r
rogers-b
ruscitti-k
sa